In [ ]:
import xarray as xr
import datetime as dt
import pandas as pd
import numpy as np
adir_adcp = 'F:/data/cruise_data/saildrone/baja-2018/adcp_data/'
adir = 'F:/data/cruise_data/saildrone/baja-2018/'

filename_loggers_usv=adir + 'saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1_withloggers.nc'
filename_adcp = adir_adcp + 'saildrone-gen_4-baja_2018-sd1002-adcp-20180411T175730-20180611T235730_300s-v1.nc'

ds_usv = xr.open_dataset(filename_loggers_usv)  #1 minute averaging
ds_usv.close()
ds_adcp = xr.open_dataset(filename_adcp)            #10 minute sampling
ds_adcp.close()
offset=0
#print(ds_adcp)
ds_usv = ds_usv.sel(trajectory=1002)
ds_adcp = ds_adcp.isel(trajectory=0)
#ds_adcp=ds_adcp.rename({'latitude':'lat','longitude':'lon'})
#ds_adcp=ds_adcp.swap_dims({'obs':'time'})
#ds_usv=ds_usv.rename({'latitude':'lat','longitude':'lon'})
ds_usv=ds_usv.swap_dims({'obs':'time'})
ds_adcp=ds_adcp.swap_dims({'obs':'time'})
lon = ds_usv.longitude
lat = ds_usv.latitude
#print(lon)
del ds_usv['longitude']
del ds_usv['latitude']
ds_usv['longitude'] = (('time'), lon)
ds_usv['latitude'] = (('time'), lat)   
#print(ds_usv)
#print(ds_adcp)

In [ ]:
ds_usv.sea_water_temperature_logger1_mean

In [ ]:
#print(ds_adcp.time[0:10].data)
#print(ds_usv.time[0,0:10].data)
#print(type(ds_adcp.time))
#ds_adcp

In [ ]:
#ds_usv2=ds_usv.sel(trajectory=1002)

#ds_usv2.time
#ds_usv2.coords('time',ds_usv2('time'))
#ds_temds.coords['day'] = ('time', [6, 7, 8])
#ds_usv2.assign_coords(obs=ds_usv2.time)
#ds_usv2

In [ ]:
tdim=ds_adcp.time[:].shape
tdim
print(ds_adcp.time)

In [ ]:
ds_adcp.longitude

In [ ]:
istart=0
print(tdim[0])
for i in range(0,tdim[0]):
    center_time=ds_adcp.time[i]
    upper_time=center_time+np.timedelta64(150,'s')
    lower_time=center_time-np.timedelta64(150,'s')
    #print(lower_time.data,upper_time.data)
    ds_usv2=ds_usv.where((ds_usv.time>lower_time) & (ds_usv.time<=upper_time)).mean(keep_attrs=True) #,upper_time))
    ds_usv2['time']=ds_adcp.time[i]
    if istart == 0:
        ds_usv3=ds_usv2
        istart=1
    else:
        ds_usv3=xr.concat([ds_usv3, ds_usv2], 'obs')

In [ ]:
ds_usv3

In [ ]:
filename_10min=adir + 'saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-10_minutes-v1_withloggers_matchedadcp_v3.nc'
ds_usv3.to_netcdf(filename_10min)

In [ ]:
#pt = pd.to_datetime(ds_usv2.time.data)+offset  #add offset seconds
#psec=pt.second
#tem=ds_usv2 #[(psec <=12)]  #data average should be only use sec 54 to sec 06 of each minute 
#tem['time']=tem['time']+offset #resample goes from :00 to :59 so shift +6sec
#print(tem.time)
#xf_avg1 = tem.resample(time='10min').mean()
filename_10min=adir + 'saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-10_minutes-v1_withloggers_matchedadcp_v2.nc'
ds_usv3=xr.open_dataset(filename_10min)

In [ ]:
ds_usv3